## The Pohlig Hellman algorithm Python Implementation

In [1]:
import math
# format: b = (a ^ x) mod p
a = 13
p = 19
b = 6

Step 1: Factor P-1

In [2]:
import os
# brute prime generator
def brutePrime():
    path = "../cachedPrimes.csv"
    if os.path.isfile(path):
        f = open(path,"r")
        data = f.read().split(",")[:-1]
        for i in range(len(data)):
            data[i] = int(data[i])
        f.close()
        return data
    else:
        f = open(path,"w")
        primes = []
        for i in range(2, 10000000):
            for j in range(2, int(i/2+1)):
                if(i % j == 0):
                    break; 
            else:
                print(i, "is prime!")
                primes.append(i)
                f.write(f"{i},")
        f.close()
        return primes
primeNums = brutePrime()

def factorNum(composite):
    factors = []
    while composite != 1:
        for i in primeNums:
            if(i > composite):
                break
            if(composite % i == 0):
                factors.append(i)
                composite /= i
                break
        else:
            print("not enough calculated primes maybe")
            return []
    return factors


In [3]:
# factor p-1
pFactors = factorNum(p-1)
print(pFactors)
# convert into hashmap to keep a count of how many of each power shows up
qs = dict()
items = list(set(pFactors))
# init the set with 0 for all the different values un the pfactors list
for i in items:
    qs[i] = 0
# set the count to the correct amount for the items
for i in pFactors:
    qs[i] += 1
print(qs)

[2, 3, 3]
{2: 1, 3: 2}


Step 2: for each q value, compute the x in the formula sum i = 0 -> amount : x = (q^i) * x_i where x_i is b^((p-1)/q) = a^((p-1/q) * x_i)

In [4]:
alpha = a
beta = b
prime = p
xis = []
xs = []
mods = []
for q in qs.keys():
    x = 0
    xit = []
    powerR = ((prime-1) / q)
    beta = b
    xi = 0
    for i in range(qs[q]):
        xi = 0
        if(i > 0):
            # this is the alpha term where temp is the inverse of alpha and beta is the new beta for the next calculation
            temp = pow(alpha, (-1 * xit[len(xit)-1]), prime)
            beta = pow(int(beta*temp), 1, prime)
        qPow = q ** i
        power = ((prime-1) / q ** (i+1))
        left = pow(int(beta),int(power), prime)
        for guess in range(prime):
            if(left == pow(int(alpha),int(powerR * guess), prime)):
                xit.append(guess)
                xi = guess
                break
        x += qPow * xi
    xs.append(x)
    xis.append(xit)


# now we need to get a list of the modululi


for key, value in qs.items():
    mods.append(key ** value)

print(xs)
print(mods)

[0, 1]
[2, 9]


Step 3: use x values alongside the chinese remainder theorem to compute the result

In [5]:
def inverseCalc(N, mod):
    N %= mod
    for inv in range(mod):
        if(((N * inv) % mod) == 1):
            return inv
        
    else:
        return -1

def crt(vals, mods):
    a = vals
    m = mods
    N = []
    invN = []
    M = 1
    for i in m:
        N.append(1)
    for i in range(len(m)):
        M *= m[i]
        for j in range(len(m)):
            if(j != i):
                N[i] *= m[j]
                
    for i in range(len(N)):
        invN.append(inverseCalc(N[i], m[i]))
    print(a)
    print(m)
    print(N)
    print(invN)
    print(M)
    
    result = 0;
    for i in range(len(a)):
        result += a[i] * N[i] * invN[i]
    
    result %= M
    return result
                
    

In [6]:
answer = crt(xs, mods)

[0, 1]
[2, 9]
[9, 2]
[1, 5]
18


In [7]:
print(f"Hence: {b}={a}^{answer} (mod {p})")
print(f"x = {answer}")

Hence: 6=13^10 (mod 19)
x = 10
